<a href="https://colab.research.google.com/github/poojan14/Mammogram_mass/blob/master/mass_detect_mammogram_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id':'14RJxFHohNu1KBc4osO_LsRo52ykIyPFj'})
download.GetContentFile('mammographic_masses.data.csv')

In [0]:
import pandas as pd
df = pd.read_csv('mammographic_masses.data.csv')

In [0]:
df.head()

,5,67,3,5.1,3.1,1
0,4,43,1,1,?,1
1,5,58,4,5,3,1
2,4,28,1,1,3,0
3,5,74,1,5,?,1
4,4,65,1,?,3,0


In [0]:
df=pd.read_csv('mammographic_masses.data.csv',na_values=['?'],names=['BI-RADS','age','shape','margin','density','severity'])
df.head()

,BI-RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [0]:
df.describe()

,BI-RADS,age,shape,margin,density,severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [0]:
df.loc[(df['age'].isnull()) | (df['shape'].isnull()) | (df['margin'].isnull()) | (df['density'].isnull())]

,BI-RADS,age,shape,margin,density,severity
1,4.0,43.0,1.0,1.0,NaN,1
4,5.0,74.0,1.0,5.0,NaN,1
5,4.0,65.0,1.0,NaN,3.0,0
6,4.0,70.0,NaN,NaN,3.0,0
7,5.0,42.0,1.0,NaN,3.0,0
...,...,...,...,...,...,...
778,4.0,60.0,NaN,4.0,3.0,0
819,4.0,35.0,3.0,NaN,2.0,0
824,6.0,40.0,NaN,3.0,4.0,1
884,5.0,NaN,4.0,4.0,3.0,1


In [0]:
df.dropna(inplace=True)
df.describe()

,BI-RADS,age,shape,margin,density,severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,4.393976,55.781928,2.781928,2.813253,2.915663,0.485542
std,1.888371,14.671782,1.242361,1.567175,0.350936,0.500092
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [0]:
all_features=df[['age','shape','margin','density']].values
all_classes=df['severity'].values
features_names=['age','shape','margin','density']
all_features

array([[67.,  3.,  5.,  3.],
       [58.,  4.,  5.,  3.],
       [28.,  1.,  1.,  3.],
       ...,
       [64.,  4.,  5.,  3.],
       [66.,  4.,  5.,  3.],
       [62.,  3.,  3.,  3.]])

In [0]:
from sklearn import preprocessing 
scaler=preprocessing.StandardScaler()
all_features_scaled=scaler.fit_transform(all_features)
all_features_scaled


array([[ 0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [ 0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [ 0.42406719,  0.17563638,  0.11923341,  0.24046607]])

In [0]:
#set up MLP model
from keras.layers import Dense
from keras.models import Sequential

def create_model():
  model=Sequential()
  #feature_input going into 6 unit layer
  model.add(Dense(6,input_dim=4,kernel_initializer='normal',activation='relu'))
  #hidden layer not important
  #model.add(Dense(4,kernel_initializer='normal',activation='relu'))
  #model.add(Dense(4,kernel_initializer='normal',activation='relu'))
  #output layer with binary classification
  model.add(Dense(1,kernel_initializer='normal',activation='relu'))
  model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
  return model

In [0]:
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier

estimator=KerasClassifier(build_fn=create_model,nb_epoch=100,verbose=2)
cv_scores=cross_val_score(estimator,all_features_scaled,all_classes,cv=20)
cv_scores.mean()

Epoch 1/1
 - 4s - loss: 2.4333 - acc: 0.5165
Epoch 1/1
 - 4s - loss: 2.2270 - acc: 0.5178
Epoch 1/1
 - 4s - loss: 2.0324 - acc: 0.5127
Epoch 1/1
 - 4s - loss: 2.0533 - acc: 0.5089
Epoch 1/1
 - 4s - loss: 3.4276 - acc: 0.5178
Epoch 1/1
 - 4s - loss: 2.0010 - acc: 0.5140
Epoch 1/1
 - 4s - loss: 2.5017 - acc: 0.5178
Epoch 1/1
 - 4s - loss: 2.1633 - acc: 0.5190
Epoch 1/1
 - 5s - loss: 2.2365 - acc: 0.5178
Epoch 1/1
 - 5s - loss: 2.2486 - acc: 0.5140
Epoch 1/1
 - 4s - loss: 1.8509 - acc: 0.5095
Epoch 1/1
 - 4s - loss: 2.8133 - acc: 0.5146
Epoch 1/1
 - 4s - loss: 2.3498 - acc: 0.5082
Epoch 1/1
 - 5s - loss: 4.1494 - acc: 0.5222
Epoch 1/1
 - 4s - loss: 2.4330 - acc: 0.5070
Epoch 1/1
 - 4s - loss: 3.2004 - acc: 0.5146
Epoch 1/1
 - 4s - loss: 1.9804 - acc: 0.5171
Epoch 1/1
 - 4s - loss: 2.2781 - acc: 0.5108
Epoch 1/1
 - 4s - loss: 2.3105 - acc: 0.5196
Epoch 1/1
 - 4s - loss: 2.1002 - acc: 0.5095


0.5147212563448181

# **100 epochs**
With 0 hidden acc= 51.44% with 1 hidden layer accuracy decreases, in udemy frank video same code accuracy = 80.361(for relu activation, rmsprop optimizer)

with no hidden in sigmoid activation and adam optimizer accuracy is 48.07%, with 1 hidden acc=51.44% (relu in hidden) with 2 hidden acc=50.12


